# Coconut tree centroid detection using U-Net

<br><br>

In [1]:
'''importing libraries'''

import keras
from keras.models import Model
from keras.layers import Input, Conv2D, MaxPooling2D, Conv1D, Concatenate, Conv2DTranspose, GaussianNoise, Lambda, Dropout, Flatten, Dense, Reshape, TimeDistributed, Permute, Softmax, Multiply, BatchNormalization, UpSampling2D
from keras import backend as K
import tensorflow as tf
from keras.models import model_from_json
tf.config.run_functions_eagerly(True)

import glob
from random import shuffle, randint
import numpy as np
from matplotlib import pyplot as plt
import imageio
import json

### 1) Reading Data

In [2]:
#接受一个参数作为高斯分布的范围，然后生成一个以均值为0，标准差为给定范围一半的二维高斯分布数组
def __getGaussian(radius):
    x = np.linspace(-radius, +radius, radius*2)
    y = np.linspace(-radius, +radius, radius*2)
    xx, yy = np.meshgrid(x, y)

    d = np.sqrt(xx**2+yy**2)
    sigma, mu = radius/2, 0.0
    gauss = np.exp(-( (d-mu)**2 / ( 2.0 * sigma**2 ) ) )
    gauss = ( gauss - np.min(gauss) ) / ( np.max(gauss) - np.min(gauss) ) # scalling between 0 to 1

    return gauss

#将给定的点列表表示的位置信息转换为图像表示，其中每个点被表示为一个高斯分布的区域
def centroids2Images(point_list, im_num_row, im_num_col, g_radius=20):

    circle_mat = __getGaussian(g_radius)

    temp_im = np.zeros((im_num_row+g_radius*2, im_num_col+g_radius*2))

    for one_pnt in point_list:
        pnt_row = int(one_pnt[0])
        pnt_col = int(one_pnt[1])

        current_patch = temp_im[g_radius+pnt_row-g_radius:g_radius+pnt_row+g_radius, g_radius+pnt_col-g_radius:g_radius+pnt_col+g_radius]
        temp_im[g_radius+pnt_row-g_radius:g_radius+pnt_row+g_radius, g_radius+pnt_col-g_radius:g_radius+pnt_col+g_radius] = np.maximum(current_patch, circle_mat)

    temp_im = temp_im[g_radius:-g_radius, g_radius:-g_radius]

    return temp_im

In [1]:
'''reading training data'''
in_h = 1280 # model input height
in_w = 1280 # model input width
imList = glob.glob("./data/TongoCoconutTree/train/*.tiff")

x_train = np.zeros((len(imList), in_h, in_w, 3)).astype('float32')
y_train = np.zeros((len(imList), in_h, in_w, 1)).astype('float32')

for i1 in range(len(imList)):
    x_train[i1,:,:,:] = imageio.imread(imList[i1])
    
    with open(imList[i1].replace('train', 'train_labels').replace('tiff', 'json')) as f:
        ctr_list = json.load(f)['centroids']
        ctr_img = centroids2Images(ctr_list, im_num_row=in_h, im_num_col=in_w, g_radius=40)
        
        y_train[i1,:,:,0] = ctr_img
        
print(x_train.shape)
print(y_train.shape)

In [2]:
'''visualizing random input/output training images'''

rdm_idx = randint(0, len(imList))

plt.figure(figsize=(10, 10))
plt.imshow(x_train[0,:,:,:].astype('uint8'))
plt.show()

plt.figure(figsize=(10, 10))
plt.imshow(y_train[0,:,:,0])
plt.show()

In [3]:
'''reading val data'''

imList = glob.glob("./data/TongoCoconutTree/val/*.tiff")

x_val = np.zeros((len(imList), in_h, in_w, 3)).astype('float32')
y_val = np.zeros((len(imList), in_h, in_w, 1)).astype('float32')

for i1 in range(len(imList)):
    x_val[i1,:,:,:] = imageio.imread(imList[i1])
    
    with open(imList[i1].replace('val', 'val_labels').replace('tiff', 'json')) as f:
        ctr_list = json.load(f)['centroids']
        ctr_img = centroids2Images(ctr_list, im_num_row=in_h, im_num_col=in_w, g_radius=40)
        y_val[i1,:,:,0] = ctr_img
        
print(x_val.shape)
print(y_val.shape)

In [4]:
'''visualizing random input/output val images'''

rdm_idx = randint(0, len(imList))

plt.figure(figsize=(10, 10))
plt.imshow(x_val[0,:,:,:].astype('uint8'))
plt.show()

plt.figure(figsize=(10, 10))
plt.imshow(y_val[0,:,:,0])
plt.show()

### 2) Model Development and Training

In [9]:
def getModel(input_shape):

    e_in = Input(shape = input_shape)

    '''Encoder'''

    e_cnn = Conv2D(32, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(e_in)
    e_cnn = BatchNormalization()(e_cnn)
    e_cnn = Conv2D(32, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(e_cnn)
    e_cnn = BatchNormalization()(e_cnn)
    e_skip1 = Dropout(0.2)(e_cnn)
    e_cnn = MaxPooling2D(pool_size=(2, 2))(e_skip1)

    e_cnn = Conv2D(32, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(e_cnn)
    e_cnn = BatchNormalization()(e_cnn)
    e_cnn = Conv2D(32, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(e_cnn)
    e_cnn = BatchNormalization()(e_cnn)
    e_skip2 = Dropout(0.2)(e_cnn)
    e_cnn = MaxPooling2D(pool_size=(2, 2))(e_skip2)

    e_cnn = Conv2D(64, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(e_cnn)
    e_cnn = BatchNormalization()(e_cnn)
    e_cnn = Conv2D(64, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(e_cnn)
    e_cnn = BatchNormalization()(e_cnn)
    e_skip3 = Dropout(0.2)(e_cnn)
    e_cnn = MaxPooling2D(pool_size=(2, 2))(e_skip3)

    e_cnn = Conv2D(128, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(e_cnn)
    e_cnn = BatchNormalization()(e_cnn)
    e_cnn = Conv2D(128, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(e_cnn)
    e_cnn = BatchNormalization()(e_cnn)
    e_skip4 = Dropout(0.2)(e_cnn)
    e_cnn = MaxPooling2D(pool_size=(2, 2))(e_skip4)

    '''Middle Part'''

    e_cnn = Conv2D(256, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(e_cnn)
    e_cnn = BatchNormalization()(e_cnn)
    e_cnn = Dropout(0.2)(e_cnn)
    e_cnn = Conv2D(256, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(e_cnn)
    e_cnn = BatchNormalization()(e_cnn)

    '''Decoder'''

    e_cnn = UpSampling2D(size = (2,2))(e_cnn)
    e_cnn = Concatenate()([e_cnn,e_skip4])
    e_cnn = Conv2D(128, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(e_cnn)
    e_cnn = BatchNormalization()(e_cnn)
    e_cnn = Conv2D(128, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(e_cnn)
    e_cnn = BatchNormalization()(e_cnn)
    e_cnn = Dropout(0.2)(e_cnn)

    e_cnn = UpSampling2D(size = (2,2))(e_cnn)
    e_cnn = Concatenate()([e_cnn,e_skip3])
    e_cnn = Conv2D(64, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(e_cnn)
    e_cnn = BatchNormalization()(e_cnn)
    e_cnn = Conv2D(64, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(e_cnn)
    e_cnn = BatchNormalization()(e_cnn)
    e_cnn = Dropout(0.2)(e_cnn)

    e_cnn = UpSampling2D(size = (2,2))(e_cnn)
    e_cnn = Concatenate()([e_cnn,e_skip2])
    e_cnn = Conv2D(32, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(e_cnn)
    e_cnn = BatchNormalization()(e_cnn)
    e_cnn = Conv2D(32, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(e_cnn)
    e_cnn = BatchNormalization()(e_cnn)
    e_cnn = Dropout(0.2)(e_cnn)

    e_cnn = UpSampling2D(size = (2,2))(e_cnn)
    e_cnn = Concatenate()([e_cnn,e_skip1])
    e_cnn = Conv2D(32, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(e_cnn)
    e_cnn = BatchNormalization()(e_cnn)
    e_cnn = Conv2D(32, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(e_cnn)
    e_cnn = BatchNormalization()(e_cnn)
    e_cnn = Dropout(0.2)(e_cnn)

    '''Last Part'''

    e_cnn = Conv2D(32, 1, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(e_cnn)
    e_cnn = BatchNormalization()(e_cnn)
    e_cnn = Conv2D(32, 1, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(e_cnn)
    e_cnn = BatchNormalization()(e_cnn)

    e_out = Conv2D(1, 1, activation = 'sigmoid')(e_cnn)

    model = Model(inputs = e_in, outputs = e_out)

    model.compile(optimizer = 'adam', loss = 'mean_squared_error')

    return model

In [5]:
'''building U-Net model'''

model = getModel(input_shape=(in_h,in_w,3))

model.summary()

In [6]:
'''fitting the model'''

batch_size = 1
epochs = 10

fit_h = model.fit(x_train, y_train, batch_size=batch_size, epochs=epochs, verbose=1, shuffle=1, validation_data=(x_val, y_val))

In [7]:
'''plotting loss curves'''

plt.plot(fit_h.history['loss'])
plt.plot(fit_h.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper right')
plt.show()


In [11]:
'''saving the model'''

model.save_weights("./models/model.h5")

np.savetxt('./models/TongoCoconutTree_loss.txt', fit_h.history['loss'], fmt='%.8f')
np.savetxt('./models/TongoCoconutTree_loss_val.txt', fit_h.history['val_loss'], fmt='%.8f')

### 3) Inference

In [8]:
'''reading test data'''

imList = glob.glob("./data/TongoCoconutTree/test/*.tiff")

x_test = np.zeros((len(imList), in_h, in_w, 3)).astype('float32')
y_test = np.zeros((len(imList), in_h, in_w, 1)).astype('float32')

for i1 in range(len(imList)):
    x_test[i1,:,:,:] = imageio.imread(imList[i1])
    
    with open(imList[i1].replace('test', 'test_labels').replace('tiff', 'json')) as f:
        ctr_list = json.load(f)['centroids']
        ctr_img = centroids2Images(ctr_list, im_num_row=in_h, im_num_col=in_w, g_radius=40)
        y_test[i1,:,:,0] = ctr_img
        
print(x_test.shape)
print(y_test.shape)

In [18]:
'''loading model'''
#select one
model.load_weights("./models/model.h5")#load self-trained model


or load a model provided by class

In [ ]:
'''loading model'''
#or load provided model by
# model.load_weights("./models/TongoCoconutTree.h5")

In [9]:
'''inference'''

p_test = model.predict(x_test, batch_size = 1)
print(p_test.shape)

In [10]:
'''visualizing random input/output/prediction images'''

rdm_idx = randint(0, len(imList))

plt.figure(figsize=(10, 10))
plt.imshow(x_test[rdm_idx,:,:,:].astype('uint8'))
plt.show()

plt.figure(figsize=(10, 10))
plt.imshow(y_test[rdm_idx,:,:,0])
plt.show()

plt.figure(figsize=(10, 10))
plt.imshow(p_test[rdm_idx,:,:,0])
plt.show()

In [21]:
plt.imsave( 'sample3_rgb.png', x_test[rdm_idx,:,:,:].astype('uint8') )
plt.imsave( 'sample3_gr.png', y_test[rdm_idx,:,:,0] )
plt.imsave( 'sample3_pred.png', p_test[rdm_idx,:,:,0] )

## Water Segmentation based on Deeplabv3

The dataset used in this notebook is a collection of water bodies images captured by the Sentinel-2 Satellite. Each image comes with a black and white mask where white represents water. The masks were created to detect and measure vegetation in satellite images.

In this notebook, I applied the pretrained DeepLabV3 ResNet-50 model in Pytorch to perform segmentation on the water body images. I chose the DeepLabV3 semantic segmentation architecture because of its effectiveness and simplicity.

**1. Prepare Problem**

In [11]:
# a) Load libraries
import os
import glob
import numpy as np
from numpy import asarray
import matplotlib.pyplot as plt
import cv2
#!pip install imagehash
import imagehash

import torchvision.models.segmentation
import torch
import torchvision.transforms as tf

from sklearn.model_selection import train_test_split
from PIL import Image

from pickle import dump

# Check if GPU parallel computing is available
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
print(device)

In [12]:
# b) Prepare dataset
# Here I selected large images height and width to improve training performance
import glob
height = width = 500 
batch = 11

images_list = sorted(glob.glob("./data\Water Bodies Dataset\Water Bodies Dataset\Images/*.jpg"))
masks_list = sorted(glob.glob("./data\Water Bodies Dataset\Water Bodies Dataset\Masks/*.jpg"))

print(len(images_list), len(masks_list))

In total, there are 2,841 photos in each Images and Masks folders.

**2. Exploratory Data Analysis**

In [13]:
# a) Image visualization
# plot first few images in Images and Masks folder 
f, axr = plt.subplots(8, 2, figsize=(12, 36)) 
for i in range(8):
    idx = np.random.randint(0, len(images_list))
    original = cv2.imread(images_list[idx])
    mask = cv2.imread(masks_list[idx])
    axr[i,0].imshow(original)
    axr[i,1].imshow(mask, cmap = 'gray')
    i +=1
    axr[0, 0].set_title("IMAGE")
    axr[0, 1].set_title("MASK")

We can see images and masks come with different shapes and even blank masks. These factors may present challenges during the transformation and training processes.

Let's examine the statistical figures of the dataset to understand how the dimensions are distributed among the images. This will help us identify any abnormal size images that can be filtered out to improve training performance.

In [ ]:
# Calculate statistics of the image dimensions
dimen_img_list = []

for img in images_list:
    img = cv2.imread(img, cv2.COLOR_BGR2RGB)
    dimen_img = img.shape[:2]
    dimen_img_list.append(dimen_img)

# Convert the list to numpy array
dimen_img_array = np.array(dimen_img_list)   

In [14]:
# Calculate statistics of the image dimensions
print("Statistics of image dimensions:")
print("Minimum width:", np.min(dimen_img_array[:, 1]))
print("Maximum width:", np.max(dimen_img_array[:, 1]))
print("Mean width:", np.mean(dimen_img_array[:, 1]))
print("Median width:", np.median(dimen_img_array[:, 1]))
print("Standard deviation of width:", np.std(dimen_img_array[:, 1]))
print("Minimum height:", np.min(dimen_img_array[:, 0]))
print("Maximum height:", np.max(dimen_img_array[:, 0]))
print("Mean height:", np.mean(dimen_img_array[:, 0]))
print("Median height:", np.median(dimen_img_array[:, 0]))
print("Standard deviation of height:", np.std(dimen_img_array[:, 0]))

In [15]:
# Plot the histogram
plt.hist(dimen_img_array[:, 1], bins=50, alpha=0.5, color='blue', label='width')
plt.hist(dimen_img_array[:, 0], bins=50, alpha=0.5, color='red', label='height')
plt.xlabel('Dimensions')
plt.ylabel('Frequency')
plt.title('Histogram of Image Dimensions')
plt.legend()
plt.show()

The statistical result suggests that the median values of height and width of the images are around 300 pixels and therefore, it would be advisable to resize all images and masks to this size or larger to ensure good training performance. To filter out any images with abnormal sizes, a cut-off threshold of 32 pixels (which is 10% of the median value) will be applied.

**3. Prepare Data**

In [16]:
# a) Data Cleaning

#Detect duplicate images and masks
hashes = {}
to_remove = []

for file in images_list:
    if file.endswith('.jpg'):
        with open(file, 'rb') as f:
            img = Image.open(f)
            # Compute the hash value for the image
            h = imagehash.phash(img)
            
            # Check if the hash value already exists in the dictionary
            if h in hashes:
                print(f'Duplicate image found: {file} and {hashes[h]}')
                to_remove.append(file)
                mask_file = os.path.join("./data/Water Bodies Dataset/Water Bodies Dataset/Masks", os.path.basename(file))
                to_remove.append(mask_file)
            else:
                hashes[h] = file


After removing 17 duplicate images and their corresponding masks, new lists of images and masks were created for the next stage of data processing.

In [17]:
# Create a new list of filenames that excludes the duplicates
new_images_list = [file for file in images_list if file not in to_remove]
new_masks_list = [os.path.join("./data/Water Bodies Dataset/Water Bodies Dataset/Masks", os.path.basename(file)) for file in new_images_list]
print(len(new_images_list), len(new_masks_list))

In [18]:
import cv2
import numpy as np

min_size = 32
df_images = []
df_masks = []

for img, mask in zip(new_images_list, new_masks_list):
    # Load images in RGB and grayscale
    n = cv2.imread(img, cv2.COLOR_BGR2RGB)
    m = cv2.imread(mask, cv2.IMREAD_GRAYSCALE)
    
    # Check if the image and mask exist and meet the size and content criteria
    if n is not None and m is not None and min(n.shape[:2]) > min_size and ((m != 0).any() and (m != 255).any()):
        # Resize images and masks to a consistent shape (e.g., 256x256)
        n = cv2.resize(n, (256, 256))  # Use desired dimensions
        m = cv2.resize(m, (256, 256))  # Use the same dimensions

        df_images.append(n)
        df_masks.append(m)

# Convert lists of images and masks to NumPy arrays
df_images = np.array(df_images)
df_masks = np.array(df_masks)

print(len(df_images), len(df_masks))


In total, 126 images and their corresponding masks were removed from the dataset, leaving a total of 2698 samples for training, testing, and validation.

In [ ]:
# Create a preprocessing pipeline for images and masks
tfImg = tf.Compose([
        tf.ToPILImage(),
        tf.Resize((height, width)),
        tf.ToTensor(),
        tf.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]),
        ])
tfMsk = tf.Compose([
        tf.ToPILImage(),
        tf.Resize((height, width)),
        tf.ToTensor(),
        ])

In [ ]:
# Create a function to read images and masks randomly
def ReadImage(df_images, df_masks):
    idx = np.random.randint(0, len(df_images))
    Img = tfImg(df_images[idx])
    Msk = tfMsk(df_masks[idx])
    return Img, Msk

In [ ]:
# Create a function to load images and masks in batch
def LoadBatch(df_images, df_masks):
    images = torch.zeros([batch, 3, height, width])
    masks = torch.zeros([batch, height, width])
    for i in range(batch):
        images[i], masks[i] = ReadImage(df_images, df_masks)
    return images, masks

In [19]:
# a) Split data into train, test and validation sets

X = df_images
y = df_masks

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.25, random_state=1)
X_test, X_val, y_test, y_val = train_test_split(X_test, y_test, test_size=0.015, random_state=1)
print(X_train.shape, y_train.shape, X_test.shape, y_test.shape, X_val.shape, y_val.shape)

**4. Evaluate Models**

In [20]:
# a) Define pretrained ResNet-50 model using TorchVision DeepLabV3 architecture
model = torchvision.models.segmentation.deeplabv3_resnet50(pretrained = True)

# Since the segmented images have 2 classes (black and white), we change the final layer to 2 classes
model.classifier[4] = torch.nn.Conv2d(256, 2, kernel_size = (1, 1), stride = (1, 1)) 
model = model.to(device)

# Select Stochastic gradient descent (SGD) optimizer with a learning rate of 0.01 (optimum for image segmentation)
optimizer = torch.optim.SGD(model.parameters(), lr=0.01)

In [ ]:
# Define loss function
criterion = torch.nn.CrossEntropyLoss() 
Losses = []

In [21]:
# b) Evaluation of train dataset
epochs = 10
best_loss = float('inf')  # Initialize best_loss to a very high value

# Training loop
for itr in range(epochs):
   images, masks = LoadBatch(X_train, y_train)
   # Load images and masks tensors to the GPU 
   images = torch.autograd.Variable(images,requires_grad = False).to(device) 
   masks = torch.autograd.Variable(masks, requires_grad = False).to(device) 
   
   # Forward training loop
   Pred = model(images)['out']
   model.zero_grad()
   Loss = criterion(Pred, masks.long())
   Losses.append(Loss.item())

   # Backward training loop
   Loss.backward() 
   optimizer.step() # Apply gradient descent to optimize weights

   # Check if the current loss is lower than the best recorded loss
   current_loss = Loss.item()
   if current_loss < best_loss:
      best_loss = current_loss
      # Save the model
      torch.save(model.state_dict(), 'best_model.pth')
      print(f"Saved better model with loss {current_loss}")

   seg = torch.argmax(Pred[0], 0).cpu().detach().numpy() # Move the tensor to the CPU, detach it from the graph, and convert it to a NumPy array
   print(itr,") Loss=", Loss.data.cpu().numpy())

In [22]:
# Plot the training loss values
plt.plot(Losses)
plt.xlabel('Iteration')
plt.ylabel('Loss')
plt.title('Training Loss')
plt.show()

The results of training the model show that the loss values were slightly better than the TensorFlow model (0.12 vs. 0.34). It seems that the optimal number of epochs for achieving good loss values is around 1000, and increasing the number of epochs beyond this point does not significantly improve training performance. 

Now that the model has been trained, we can proceed to testing it with new images.

**5. Finalize Model**

In [ ]:
#Load the trained model weights by yourself
model.load_state_dict(torch.load(r'D:\gxy\01_lesson\00use_Deep-Learning-Projects-main\Deep-Learning-Projects-main\best_model.pth'))
# Load the pretrained optimal model weights
#model.load_state_dict(torch.load(r'./models/water/water_best_model_epoch1000.pth'))
# Set the model to evaluation mode
model.eval()

In [23]:
# Set trained model to the evaluation mode for validation test set
model.eval() 

In [24]:
Losses_test = []

with torch.no_grad(): # Set context to not calculate and save gradients
  correct = 0
  total = 0
  for itr in range(epochs):
   images_test, masks_test = LoadBatch(X_test, y_test)
   # Load images and masks tensors to the GPU 
   images_test = torch.autograd.Variable(images_test,requires_grad = False).to(device) 
   masks_test = torch.autograd.Variable(masks_test, requires_grad = False).to(device) 
   
   Pred_test = model(images_test)['out']
   model.zero_grad()
   Loss_test = criterion(Pred_test, masks_test.long())
   Losses_test.append(Loss_test.item())

   seg = torch.argmax(Pred_test[0], 0).cpu().detach().numpy() # Move the tensor to the CPU, detach it from the graph, and convert it to a NumPy array
   print(itr,") Loss=", Loss_test.data.cpu().numpy())

In [25]:
# Plot the training loss values
plt.figure(figsize = (10, 5))
plt.plot(Losses, label = 'training')
plt.plot(Losses_test, label = 'testing')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()

The consistency between the training and validation losses suggests that the model is capable of accurately predicting on new data.

In [ ]:
# a) Predictions on validation data
Pred_val = []
with torch.no_grad(): # Set context to not calculate and save gradients
    for i in X_val:
        Img_tensor = tfImg(i)
        # Load tensor images to GPU    
        Img_tensor = torch.autograd.Variable(Img_tensor, requires_grad = False).to(device).unsqueeze(0)
        Pred2 = model(Img_tensor)['out']
        Pred_val.append(Pred2)

In [ ]:
pred_np_list = []
for i in range(len(Pred_val)):
    #Get original shape of images in X_val
    height_orgin, width_orgin, d = X_val[i].shape 
    pred_tensor = Pred_val[i]
    # Resize segmented images to origninal sizes
    pred_resized = tf.Resize((height_orgin,width_orgin))(pred_tensor[0]) 
    #Convert the tensor image to a numpy array
    pred_np = pred_resized.cpu().detach().numpy()
    pred_np_list.append(pred_np)

In [27]:
# Plot original and predicted images
fig, ax = plt.subplots(len(X_val), 3, figsize=(12,40))
for i in range(len(X_val)):
    orig = X_val[i]
    msk = y_val[i]
    seg = pred_np_list[i][1, :, :]
    ax[i,0].imshow(orig)
    ax[i,1].imshow(msk, cmap='gray')
    ax[i,2].imshow(seg, cmap='gray')
    i +=1
    ax[0, 0].set_title("IMAGE")
    ax[0, 1].set_title("MASK")
    ax[0, 2].set_title("PREDICTION")

**6. Conclusions**

- Data cleaning is essential to remove duplicate, blank and abnormal size images.

- Increasing the size of the images, batch size, the number of epochs and size of dataset can enhance the performance of the training process.

- The DeepLabV3 architecture is a simple and highly effective pretrained model for semantic segmentation in pytorch.

- When comparing the validation loss of the pretrained Resnet-50 model in Pytorch to that of the Keras model (0.12 vs. 0.34), we can see the computational demands of the DeepLabV3 model are significantly higher but its prediction accuracy is about the same as the Keras model.

## Road Segmentation based on Deeplabv3+

In [ ]:
import os, cv2
import numpy as np
import pandas as pd
import random, tqdm
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

import warnings
warnings.filterwarnings("ignore")

import torch
import torch.nn as nn
from torch.utils.data import DataLoader
import albumentations as album

In [28]:
# !pip install -q -U segmentation-models-pytorch albumentations > /dev/null
import segmentation_models_pytorch as smp
print(dir(smp))

### Read Data & Create train / valid splits 

In [29]:
DATA_DIR = './data/Road Dataset/'

metadata_df = pd.read_csv(os.path.join(DATA_DIR, 'metadata.csv'))
metadata_df = metadata_df[metadata_df['split']=='train']
metadata_df = metadata_df[['image_id', 'sat_image_path', 'mask_path']]
metadata_df['sat_image_path'] = metadata_df['sat_image_path'].apply(lambda img_pth: os.path.join(DATA_DIR, img_pth))
metadata_df['mask_path'] = metadata_df['mask_path'].apply(lambda img_pth: os.path.join(DATA_DIR, img_pth))
# Shuffle DataFrame
metadata_df = metadata_df.sample(frac=1).reset_index(drop=True)

# Perform 90/10 split for train / val
valid_df = metadata_df.sample(frac=0.1, random_state=42)
train_df = metadata_df.drop(valid_df.index)
len(train_df), len(valid_df)

In [30]:
class_dict = pd.read_csv(os.path.join(DATA_DIR, 'class_dict.csv'))
# Get class names
class_names = class_dict['name'].tolist()
# Get class RGB values
class_rgb_values = class_dict[['r','g','b']].values.tolist()

print('All dataset classes and their corresponding RGB values in labels:')
print('Class Names: ', class_names)
print('Class RGB values: ', class_rgb_values)

#### Shortlist specific classes to segment

In [31]:
# Useful to shortlist specific classes in datasets with large number of classes
select_classes = ['background', 'road']

# Get RGB values of required classes
select_class_indices = [class_names.index(cls.lower()) for cls in select_classes]
select_class_rgb_values =  np.array(class_rgb_values)[select_class_indices]

print('Selected classes and their corresponding RGB values in labels:')
print('Class Names: ', class_names)
print('Class RGB values: ', class_rgb_values)

### Helper functions for viz. & one-hot encoding/decoding

In [ ]:
# helper function for data visualization
def visualize(**images):
    """
    Plot images in one row
    """
    n_images = len(images)
    plt.figure(figsize=(20,8))
    for idx, (name, image) in enumerate(images.items()):
        plt.subplot(1, n_images, idx + 1)
        plt.xticks([]); 
        plt.yticks([])
        # get title from the parameter names
        plt.title(name.replace('_',' ').title(), fontsize=20)
        plt.imshow(image)
    plt.show()

# Perform one hot encoding on label
def one_hot_encode(label, label_values):
    """
    Convert a segmentation image label array to one-hot format
    by replacing each pixel value with a vector of length num_classes
    # Arguments
        label: The 2D array segmentation image label
        label_values
        
    # Returns
        A 2D array with the same width and hieght as the input, but
        with a depth size of num_classes
    """
    semantic_map = []
    for colour in label_values:
        equality = np.equal(label, colour)
        class_map = np.all(equality, axis = -1)
        semantic_map.append(class_map)
    semantic_map = np.stack(semantic_map, axis=-1)

    return semantic_map
    
# Perform reverse one-hot-encoding on labels / preds
def reverse_one_hot(image):
    """
    Transform a 2D array in one-hot format (depth is num_classes),
    to a 2D array with only 1 channel, where each pixel value is
    the classified class key.
    # Arguments
        image: The one-hot format image 
        
    # Returns
        A 2D array with the same width and hieght as the input, but
        with a depth size of 1, where each pixel value is the classified 
        class key.
    """
    x = np.argmax(image, axis = -1)
    return x

# Perform colour coding on the reverse-one-hot outputs
def colour_code_segmentation(image, label_values):
    """
    Given a 1-channel array of class keys, colour code the segmentation results.
    # Arguments
        image: single channel array where each value represents the class key.
        label_values

    # Returns
        Colour coded image for segmentation visualization
    """
    colour_codes = np.array(label_values)
    x = colour_codes[image.astype(int)]

    return x

In [ ]:
class RoadsDataset(torch.utils.data.Dataset):

    """DeepGlobe Road Extraction Challenge Dataset. Read images, apply augmentation and preprocessing transformations.
    
    Args:
        df (str): DataFrame containing images / labels paths
        class_rgb_values (list): RGB values of select classes to extract from segmentation mask
        augmentation (albumentations.Compose): data transfromation pipeline 
            (e.g. flip, scale, etc.)
        preprocessing (albumentations.Compose): data preprocessing 
            (e.g. noralization, shape manipulation, etc.)
    
    """
    def __init__(
            self, 
            df,
            class_rgb_values=None, 
            augmentation=None, 
            preprocessing=None,
    ):
        self.image_paths = df['sat_image_path'].tolist()
        self.mask_paths = df['mask_path'].tolist()
        
        self.class_rgb_values = class_rgb_values
        self.augmentation = augmentation
        self.preprocessing = preprocessing
    
    def __getitem__(self, i):
        
        # read images and masks
        image = cv2.cvtColor(cv2.imread(self.image_paths[i]), cv2.COLOR_BGR2RGB)
        mask = cv2.cvtColor(cv2.imread(self.mask_paths[i]), cv2.COLOR_BGR2RGB)
        
        # one-hot-encode the mask
        mask = one_hot_encode(mask, self.class_rgb_values).astype('float')
        
        # apply augmentations
        if self.augmentation:
            sample = self.augmentation(image=image, mask=mask)
            image, mask = sample['image'], sample['mask']
        
        # apply preprocessing
        if self.preprocessing:
            sample = self.preprocessing(image=image, mask=mask)
            image, mask = sample['image'], sample['mask']
            
        return image, mask
        
    def __len__(self):
        # return length of 
        return len(self.image_paths)

#### Visualize Sample Image and Mask

In [32]:
dataset = RoadsDataset(train_df, class_rgb_values=select_class_rgb_values)
random_idx = random.randint(0, len(dataset)-1)
image, mask = dataset[2]

visualize(
    original_image = image,
    ground_truth_mask = colour_code_segmentation(reverse_one_hot(mask), select_class_rgb_values),
    one_hot_encoded_mask = reverse_one_hot(mask)
)

### Defining Augmentations

In [ ]:
def get_training_augmentation():
    train_transform = [
        album.HorizontalFlip(p=0.5),
        album.VerticalFlip(p=0.5),
    ]
    return album.Compose(train_transform)


def to_tensor(x, **kwargs):
    return x.transpose(2, 0, 1).astype('float32')


def get_preprocessing(preprocessing_fn=None):
    """Construct preprocessing transform    
    Args:
        preprocessing_fn (callable): data normalization function 
            (can be specific for each pretrained neural network)
    Return:
        transform: albumentations.Compose
    """
    _transform = []
    if preprocessing_fn:
        _transform.append(album.Lambda(image=preprocessing_fn))
    _transform.append(album.Lambda(image=to_tensor, mask=to_tensor))
        
    return album.Compose(_transform)

#### Visualize Augmented Images & Masks

In [33]:
augmented_dataset = RoadsDataset(
    train_df, 
    augmentation=get_training_augmentation(),
    class_rgb_values=select_class_rgb_values,
)

random_idx = random.randint(0, len(augmented_dataset)-1)

# Different augmentations on image/mask pairs
for idx in range(3):
    image, mask = augmented_dataset[idx]
    visualize(
        original_image = image,
        ground_truth_mask = colour_code_segmentation(reverse_one_hot(mask), select_class_rgb_values),
        one_hot_encoded_mask = reverse_one_hot(mask)
    )

## Training DeepLabV3+

### Model Definition

In [ ]:
ENCODER = 'resnet50'
ENCODER_WEIGHTS = 'imagenet'
CLASSES = select_classes
ACTIVATION = 'sigmoid' # could be None for logits or 'softmax2d' for multiclass segmentation

# create segmentation model with pretrained encoder
model = smp.DeepLabV3Plus(
    encoder_name=ENCODER, 
    encoder_weights=ENCODER_WEIGHTS, 
    classes=len(CLASSES), 
    activation=ACTIVATION,
)

preprocessing_fn = smp.encoders.get_preprocessing_fn(ENCODER, ENCODER_WEIGHTS)

#### Get Train / Val DataLoaders

In [ ]:
# Get train and val dataset instances
train_dataset = RoadsDataset(
    train_df, 
    augmentation=get_training_augmentation(),
    preprocessing=get_preprocessing(preprocessing_fn),
    class_rgb_values=select_class_rgb_values,
)

valid_dataset = RoadsDataset(
    valid_df, 
    preprocessing=get_preprocessing(preprocessing_fn),
    class_rgb_values=select_class_rgb_values,
)

# Get train and val data loaders
train_loader = DataLoader(train_dataset, batch_size=3, shuffle=True, num_workers=0)
valid_loader = DataLoader(valid_dataset, batch_size=3, shuffle=False, num_workers=0)

#### Set Hyperparams

In [34]:
from segmentation_models_pytorch import utils
# Set flag to train the model or not. If set to 'False', only prediction is performed (using an older model checkpoint)
TRAINING = True

# Set num of epochs
EPOCHS = 2

# Set device: `cuda` or `cpu`
#DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
DEVICE = torch.device("cpu")
print(DEVICE)
# define loss function
loss = smp.utils.losses.DiceLoss()

# define metrics
metrics = [
    smp.utils.metrics.IoU(threshold=0.5),
]

# define optimizer
optimizer = torch.optim.Adam([ 
    dict(params=model.parameters(), lr=0.00008),
])

# define learning rate scheduler (not used in this NB)
lr_scheduler = torch.optim.lr_scheduler.CosineAnnealingWarmRestarts(
    optimizer, T_0=1, T_mult=2, eta_min=5e-5,
)


In [ ]:
train_epoch = smp.utils.train.TrainEpoch(
    model, 
    loss=loss, 
    metrics=metrics, 
    optimizer=optimizer,
    device=DEVICE,
    verbose=True,
)

valid_epoch = smp.utils.train.ValidEpoch(
    model, 
    loss=loss, 
    metrics=metrics, 
    device=DEVICE,
    verbose=True,
)

### Training DeepLabV3+

In [35]:
%%time

if TRAINING:

    best_iou_score = 0.0
    train_logs_list, valid_logs_list = [], []

    for i in range(0, EPOCHS):

        # Perform training & validation
        print('\nEpoch: {}'.format(i))
        train_logs = train_epoch.run(train_loader)
        valid_logs = valid_epoch.run(valid_loader)
        train_logs_list.append(train_logs)
        valid_logs_list.append(valid_logs)

        # Save model if a better val IoU score is obtained
        if best_iou_score < valid_logs['iou_score']:
            best_iou_score = valid_logs['iou_score']
            torch.save(model, './train_model.pth')
            print('Model saved!')

### Prediction on Test Data

In [36]:
# load saved model checkpoint from the current run
#此处加载上面刚刚训练的模型，无需修改路径，运行该代码，则无需运行下一个cell
if os.path.exists('./train_model.pth'):
    model = torch.load('./train_model.pth', map_location=torch.device("cpu"))
    print('Loaded DeepLabV3+ model from current train.')

In [37]:
# load best saved model checkpoint from previous commit (if present)
# 如果使用提供之前训练好的最优模型,运行该代码
# 由于直接加载最优模型，没有训练过程将无法显示损失曲线，直接运行Prediction on Test Data部分
if os.path.exists('./models/road/road_best_model_epoch100.pth'):
    best_model = torch.load('./models/road/road_best_model_epoch100.pth', map_location=DEVICE)
    print('Loaded pre-trained DeepLabV3+ model!')
# 将模型设置为评估模式
model.eval()

In [38]:
# create test dataloader to be used with DeepLabV3+ model (with preprocessing operation: to_tensor(...))
test_dataset = RoadsDataset(
    valid_df, 
    preprocessing=get_preprocessing(preprocessing_fn),
    class_rgb_values=select_class_rgb_values,
)

test_dataloader = DataLoader(test_dataset)

# test dataset for visualization (without preprocessing augmentations & transformations)
test_dataset_vis = RoadsDataset(
    valid_df,
    class_rgb_values=select_class_rgb_values,
)

# get a random test image/mask index
random_idx = random.randint(0, len(test_dataset_vis)-1)
image, mask = test_dataset_vis[random_idx]

visualize(
    original_image = image,
    ground_truth_mask = colour_code_segmentation(reverse_one_hot(mask), select_class_rgb_values),
    one_hot_encoded_mask = reverse_one_hot(mask)
)


### Model Evaluation on Test Dataset

In [39]:

test_epoch = smp.utils.train.ValidEpoch(
    model,
    loss=loss, 
    metrics=metrics, 
    device=DEVICE,
    verbose=True,
)

valid_logs = test_epoch.run(test_dataloader)
print("Evaluation on Test Data: ")
print(f"Mean IoU Score: {valid_logs['iou_score']:.4f}")
print(f"Mean Dice Loss: {valid_logs['dice_loss']:.4f}")

### Plot Dice Loss & IoU Metric for Train vs. Val

In [40]:
train_logs_df = pd.DataFrame(train_logs_list)
valid_logs_df = pd.DataFrame(valid_logs_list)
train_logs_df.T

In [ ]:
plt.figure(figsize=(20,8))
plt.plot(train_logs_df.index.tolist(), train_logs_df.iou_score.tolist(), lw=3, label = 'Train')
plt.plot(valid_logs_df.index.tolist(), valid_logs_df.iou_score.tolist(), lw=3, label = 'Valid')
plt.xlabel('Epochs', fontsize=20)
plt.ylabel('IoU Score', fontsize=20)
plt.title('IoU Score Plot', fontsize=20)
plt.legend(loc='best', fontsize=16)
plt.grid()
plt.savefig('iou_score_plot.png')
plt.show()

In [ ]:
plt.figure(figsize=(20,8))
plt.plot(train_logs_df.index.tolist(), train_logs_df.dice_loss.tolist(), lw=3, label = 'Train')
plt.plot(valid_logs_df.index.tolist(), valid_logs_df.dice_loss.tolist(), lw=3, label = 'Valid')
plt.xlabel('Epochs', fontsize=20)
plt.ylabel('Dice Loss', fontsize=20)
plt.title('Dice Loss Plot', fontsize=20)
plt.legend(loc='best', fontsize=16)
plt.grid()
plt.savefig('dice_loss_plot.png')
plt.show()

### Prediction on Test Data

In [ ]:
sample_preds_folder = 'sample_predictions/'
if not os.path.exists(sample_preds_folder):
    os.makedirs(sample_preds_folder)

In [41]:
for idx in range(len(test_dataset)):

    image, gt_mask = test_dataset[idx]
    image_vis = test_dataset_vis[idx][0].astype('uint8')
    x_tensor = torch.from_numpy(image).to(torch.device("cpu")).unsqueeze(0)
    # 使用模型实例进行预测
    with torch.no_grad():  # 不需要计算梯度
        pred_mask = best_model(x_tensor)#使用之前训练好的最优模型
        #pred_mask = model(x_tensor)#使用现在训练的模型
    pred_mask = pred_mask.detach().squeeze().cpu().numpy()
    #pred_mask = pred_mask.detach().squeeze().cpu().numpy()
    # Convert pred_mask from `CHW` format to `HWC` format
    pred_mask = np.transpose(pred_mask,(1,2,0))
    # Get prediction channel corresponding to foreground
    pred_road_heatmap = pred_mask[:,:,select_classes.index('road')]
    pred_mask = colour_code_segmentation(reverse_one_hot(pred_mask), select_class_rgb_values)
    # Convert gt_mask from `CHW` format to `HWC` format
    gt_mask = np.transpose(gt_mask,(1,2,0))
    gt_mask = colour_code_segmentation(reverse_one_hot(gt_mask), select_class_rgb_values)
    cv2.imwrite(os.path.join(sample_preds_folder, f"sample_pred_{idx}.png"), np.hstack([image_vis, gt_mask, pred_mask])[:,:,::-1])
    
    visualize(
        original_image = image_vis,
        ground_truth_mask = gt_mask,
        predicted_mask = pred_mask,
        pred_road_heatmap = pred_road_heatmap
    )